In [1]:
import astropy.units as units
import astropy.constants as constants
import matplotlib.pyplot as plt
import sympy as sym
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import requests
import re
import subprocess
import urllib.request
from sympy.abc import *
# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"
#%matplotlib notebook #incompatible with mpmath

Error importing optional module geopandas
Traceback (most recent call last):
  File "c:\Users\engin\anaconda3\lib\site-packages\_plotly_utils\optional_imports.py", line 30, in get_module
    return import_module(name)
  File "c:\Users\engin\anaconda3\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "c:\Users\engin\anaconda3\lib\site-packages\geopandas\__init__.py", line 3, in <module>
    from geopandas.geoseries import GeoSeries  # noqa
  File "c:\Users\engin\anaconda3\lib\site-package

This function queries the International Atomic Energy Agency's Nuclear Data Services (https://nds.iaea.org/).

API: https://nds.iaea.org/relnsd/vcharthtml/api_v0_guide.html 

Example Jupyter Notebook for automating data downloads: https://www-nds.iaea.org/relnsd/vcharthtml/api_v0_notebook.html  

In [2]:
def lc_read_csv(url):
    '''
    Query the livechart service and return a pandas dataframe. 
    format: lc_read_csv(livechart + "fields=decay_rads&nuclides=" + "16O" + "&rad_types=bm")
    '''
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

 This function takes a dataframe of parent nuclides and returns a list of dataframes of daughter nuclides

In [3]:
def get_daughter_df(parent_df):
    '''
    This function takes a dataframe of parent nuclides and returns a 
    list of dataframes of daughter nuclides
    '''
    daughter_nucleus= [str(int(parent_df['d_z'][row]) + int(parent_df['d_n'][row])) + 
                    ''.join(parent_df['d_symbol'][row]) for row in range(parent_df.shape[0])]
    daughter_df = [lc_read_csv(livechart + "fields=decay_rads&nuclides=" + daughter_nucleus[row].lower() 
                    + "&rad_types=bm") for row in range(parent_df.shape[0])]
    daughter_df = pd.concat(daughter_df)
    return daughter_df


It appears that when queried for stable nuclides, livechart return a 0x1 dataframe with only the number 0

In [4]:
#It appears that when queried for stable nuclides, livechart return a 0x1 dataframe with only the number 0
lc_read_csv(livechart + "fields=decay_rads&nuclides=" + "16O" + "&rad_types=bm")

,0


In [5]:
df = lc_read_csv(livechart + "fields=decay_rads&nuclides=42ar&rad_types=bm")
df.iloc[ : , : 15]

,mean_energy,unc_me,intensity_beta,unc_ib,daughter_level_energy,max_energy,unc_me.1,log_ft,unc_lf,transition_type,anti_nu_mean_energy,unc_ame,p_z,p_n,p_symbol
0,233,16,100,0,0,599,6,9.32,0.17,1U,367,3.6,18,24,Ar


In [6]:
df.iloc[ : , 15: ]

,p_energy_shift,p_energy,unc_pe,jp,half_life,operator_hl,unc_hl,unit_hl,half_life_sec,unc_hls,...,decay_%,unc_d,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,,0,,0+,32.9,,11,Y,1.038223e+09,3.471262e+07,...,100,,599,6,19,23,K,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-10


In [7]:
float(df['mean_energy']) * float(df['intensity_beta']) / 100

233.0

In [8]:
daughter_nucleus = str(int(df['d_z']) + int(df['d_n'])) + ''.join(df['d_symbol'])
#make all letters lowercase 
daughter_nucleus.lower()

'42k'

In [9]:
daughter_df = lc_read_csv(livechart + "fields=decay_rads&nuclides=" + daughter_nucleus.lower() + "&rad_types=bm")
daughter_df.iloc[ : , : 15]

,mean_energy,unc_me,intensity_beta,unc_ib,daughter_level_energy,max_energy,unc_me.1,log_ft,unc_lf,transition_type,anti_nu_mean_energy,unc_ame,p_z,p_n,p_symbol
0,21.41,0.21,0.07,0.01,3445.4,79.80,0.70,4.9800,0.0700,A,58.60,0.60,19,23,K
1,415.41,0.2,0.05,0.01,2424.3,1100.90,0.40,9.0500,0.0900,1NU,686.70,0.60,19,23,K
2,702.95,0.2,0.34,0.03,1837.2,1688.00,0.40,9.9200,0.0400,1U,986.49,0.36,19,23,K
3,824.32,0.17,17.64,0.09,1524.6,2000.60,0.30,7.5501,0.0023,1NU,1177.16,0.25,19,23,K
4,1565.86,,81.90,0.09,0.0,3525.22,0.18,9.4768,0.0006,1U,1961.70,0.09,19,23,K


In [10]:
daughter_df.iloc[ : , 15: ]

,p_energy_shift,p_energy,unc_pe,jp,half_life,operator_hl,unc_hl,unit_hl,half_life_sec,unc_hls,...,decay_%,unc_d,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-10
1,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-10
2,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-10
3,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-10
4,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-10


In [11]:
daughter_df = get_daughter_df(df)

In [12]:
third_gen_df = get_daughter_df(daughter_df)
third_gen_df

,0


Where there 1 is one mole of the 0-th generation nucleus at $t=0$ there are $m_i$ moles of the $i-th$ generation nucleus at time $t$. We assume that the $i-th$ generation nucleus has a measurable e-folding time of $\lambda_i$. Additionally, $r_i$ is the decay rate of the $i-th$ generation nucleus in moles/time.
\begin{equation}
\begin{split}
\Large m_0 = e^{\Large\frac{t}{-\lambda_0}}\\
\Large m_{i>0} = -\frac{1}{\lambda_0}\int_0^te^{\Large t(\Large\frac{1}{ -\lambda_0}+\frac{1}{-\lambda_1})}a_{i-1}dt\\
\Large r_i = -\frac{1}{\Large\lambda_0}e^{\Large\frac{t}{-\lambda_0}}a_i\\
\Large a_i = \prod_{k=1}^{i}(1-e^{ t\Large\frac{1}{-\lambda_k}})
\end{split}
\end{equation}
We also assume an average decay energy $\bar{E}_i$ for the nuclide in each generation, and thus we have for the total power produced by the decay chain and for the power density $\rho_P$
\begin{equation}
\begin{split}
\Large P = \sum_{j=0}^N \bar{E}_ir_i\\
\Large \rho_P = \frac{\sum_{j=0}^N \bar{E}_ir_i}{\sum_{j=0}^N m_i}
\end{split}
\end{equation}
Less than $10^{-7}$ of the mass of the nuclide is lost in $\beta-$ decay so we have
\begin{equation}
\begin{split}
\Large \rho_P = \frac{1}{m_0}\sum_{j=0}^N \bar{E}_ir_i\\
\end{split}
\end{equation}

In [24]:
#Quickly calculating the decay rate of the i-th generation nuclide
def formulate_decay_rate(i, e_folding_times):
    '''
    The e-folding times must be in a numpy array.
    This function takes the  generation (i, 0-indexed) of the nuclide 
    and the e-folding times of the entire decay chain.
    Returns a formula for the decay rate
    '''
    exponent_array = -1 / e_folding_times[:i+1]
    l_0 = exponent_array[0]
    r_i = l_0 * sym.exp(t * l_0) * np.prod([1 - sym.exp(t * l) for l in exponent_array])
    return r_i

def eval_decay_rates(decay_rates, time_array):
    '''
    This function takes the formula for the decay rate of each generation
    and substitutes each value in the time array for t.
    '''
    try:
        evaluated_decay_rates = [[formula.subs(t, time) for time in time_array] 
                            for formula in decay_rates]
    except:
        evaluated_decay_rates = [decay_rates.subs(t, time) for time in time_array]
    return evaluated_decay_rates


#Calculating the power density of a decay chain
def calc_power_density(decay_rates, decay_energies, initial_mass):
    '''
    decay_rates must be a numpy array in moles/second
    decay_energies must be a numpy array in keV/decay
    This function takes the decay rates and energies of the decay chain.
    Returns the power density in watts/g
    '''
    power_density =  np.sum(decay_rates * decay_energies) * units.keV.to(units.J)
    #convert from decays to moles
    power_density *= float(constants.N_A * units.mol) / initial_mass
    return power_density

e_folding_times = np.array([10**2])
time_array = np.linspace(0, 10**2, 10**2)
decay_rates = formulate_decay_rate(0, e_folding_times)
eval_decay_rates(decay_rates, time_array)

TypeError: 'Mul' object is not iterable

In [25]:
len(5)

TypeError: object of type 'int' has no len()

In [129]:
eval_r_i = sym.lambdify(t, calc_decay_rate(1, np.array([10**9.5])))
time_array = np.logspace(-7, 2, 10**3)
decay_rates = np.absolute([(np.round(eval_r_i(t), 10)) for t in time_array]) * units.s.to(units.year)
xlabel, ylabel = "Time (years)", "Decay Rate (moles/seconds)"
decay_df = pd.DataFrame({xlabel: time_array, ylabel: decay_rates})
px.scatter(decay_df, x = xlabel, y = ylabel, log_x = True, log_y = True,
            title = "Decay Rate of chain with e-folding times " + str(e_folding_times))

In [130]:
max(decay_rates)

0.0

In [131]:
time_array

array([1.00000000e-07, 1.02096066e-07, 1.04236067e-07, 1.06420924e-07,
       1.08651577e-07, 1.10928986e-07, 1.13254132e-07, 1.15628013e-07,
       1.18051653e-07, 1.20526094e-07, 1.23052400e-07, 1.25631660e-07,
       1.28264983e-07, 1.30953502e-07, 1.33698374e-07, 1.36500781e-07,
       1.39361927e-07, 1.42283046e-07, 1.45265393e-07, 1.48310251e-07,
       1.51418933e-07, 1.54592774e-07, 1.57833141e-07, 1.61141428e-07,
       1.64519059e-07, 1.67967487e-07, 1.71488197e-07, 1.75082703e-07,
       1.78752553e-07, 1.82499324e-07, 1.86324631e-07, 1.90230119e-07,
       1.94217468e-07, 1.98288395e-07, 2.02444651e-07, 2.06688025e-07,
       2.11020343e-07, 2.15443469e-07, 2.19959307e-07, 2.24569800e-07,
       2.29276931e-07, 2.34082728e-07, 2.38989257e-07, 2.43998630e-07,
       2.49113003e-07, 2.54334576e-07, 2.59665597e-07, 2.65108360e-07,
       2.70665207e-07, 2.76338529e-07, 2.82130768e-07, 2.88044415e-07,
       2.94082017e-07, 3.00246171e-07, 3.06539530e-07, 3.12964801e-07,
      

In [49]:
def lambda_to_half_life(e_folding_time):
    '''
    This function takes the e-folding times of the entire decay chain.
    Returns the half-life of the nuclide
    '''
    return np.log(2) * e_folding_time

def half_life_to_lambda(half_life):
    '''
    This function takes the e-folding times of the entire decay chain.
    Returns the half-life of the nuclide
    '''
    return half_life / np.log(2)


\begin{equation}
\begin{split}
m = 2^{-t\frac{1}{h}}\\
m = e^{-t\frac{1}{h}\log2}\\
m = e^{-t\frac{1}{\lambda}}\\
-t\frac{1}{h}\log2 = -t\frac{1}{\lambda}\\
h = \lambda\log2
\end{split}
\end{equation}

In [18]:
delta_new, m_0, lambda_m, lambda_d = sym.symbols('\delta_{new} m_0 lambda_m lambda_delta', 
                positive = True, real = True)
delta_tau, tau = sym.symbols('delta_{tau} tau', positive = True, real = True)
lambda_t = sym.symbols('lambda')
#the probability a nucleus with a given e-folding time l after a time t
p = sym.exp(-1 * tau / lambda_d)
#the daughter nuclei being instantaneously created at a given time t
delta_new = m_0 * sym.exp(-1 * tau / lambda_m) / lambda_m
#daughter nuclei as a function of time
delta_tau = m_0 * lambda_d * (sym.exp(-1 * tau * (1/lambda_d + 1/lambda_m)) - 1) / (lambda_m + lambda_d)
delta_tau = sym.collect(delta_tau, t)

In [19]:
attempt = sym.simplify(
    sym.integrate(delta_new * p, (tau, 0, t)))
attempt

-lambda_delta*m_0*exp(-t/lambda_m - t/lambda_delta)/(lambda_delta + lambda_m) + lambda_delta*m_0/(lambda_delta + lambda_m)

In [24]:
attempt = sym.integrate(delta_new * p, (tau, 0, t))
eval_attempt = sym.lambdify(([t, lambda_m, lambda_d, m_0]), attempt, )
#specify the implementation of the function to be consistent with the input types
eval_attempt(.1, 1, 1, 1)

0.09063462346100915

In [ ]:
evaluate_delta_tau = sym.lambdify(([tau, lambda_m, lambda_d, m_0]), delta_tau)
evaluate_delta_tau(0, 1, 1, 1) #should be 0

0.0

In [ ]:
evaluate_delta_tau(1, 32.9, 12 * units.h.to(units.year), 1) 

0.029896242480243584

In [ ]:
p_t = 1 - sym.exp(-1 * t / lambda_t)
prob_not_decay = sym.lambdify(([t, lambda_t]), p_t)
def daughter_nucleus_abundance(t, lambda_d, parent_initial_mol, parent_nucleus_decay):
    m_0 = parent_initial_mol
    p = 1 - sym.exp(-1 * t / lambda_d)
    delta_tau = sym.integrate(p * parent_nucleus_decay, (t, 0, t))
    evaluate_delta_tau = sym.lambdify(([tau, lambda_m, lambda_d, m_0]), delta_tau, 'numpy')
    t, lambda_d, parent_initial_mol = np.float(t), np.float(lambda_d), np.float(parent_initial_mol)
    return evaluate_delta_tau(t, lambda_d, m_0, parent_nucleus_decay)

